# 1. 泛映射类型  
collections.abc 模块中有 Mapping 和 MutableMapping 这两个抽象基类，它们的作用是为 dict 和其他类似的类型定义形式接口  
非抽象映射类型一般不会直接继承这些抽象基类，它们会直接collections.UserDict 进行扩展。

什么是可散列的数据类型?  
如果一个对象是可散列的，那么在这个对象的生命周期中，它的散列值是不变的，而且这个对象需要实现 \__hash\__() 方法。另外可散列对象还要有\__qe\__() 方法，这样才能跟其他键做比较。如果两个可散列对象是相等的，那么它们的散列值一定是一样的……  
原子不可变数据类型（ str、 bytes 和数值类型）都是可散列类型， frozenset 也是可散列的，因为根据其定义， frozenset 里只能容纳可散列类型。元组的话，只有当一个元组包含的所有元素都是可散列类型的情况下，它才是可散列的。

# 2. 字典推导式

字典初始化方法，除了以下列出的方法还有字典推导式。

In [43]:
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

字典推导（ dictcomp）可以从任何以键值对作为元素的可迭代对象中构建出字典。

In [44]:
charorder = {char:num for num,char in zip(range(26), 'abcdefghijklmnopqrstuvwxyz')}
print(charorder)
orderchar = {num:char for num,char in zip(range(26), 'abcdefghijklmnopqrstuvwxyz')}
print(orderchar)

{'f': 5, 'a': 0, 'h': 7, 't': 19, 'q': 16, 'y': 24, 'e': 4, 'j': 9, 'k': 10, 'v': 21, 'w': 22, 'n': 13, 'o': 14, 'm': 12, 'd': 3, 's': 18, 'u': 20, 'p': 15, 'i': 8, 'g': 6, 'l': 11, 'b': 1, 'x': 23, 'z': 25, 'r': 17, 'c': 2}
{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w', 23: 'x', 24: 'y', 25: 'z'}


以上也可以看出，dict是没有顺序的

# 3. 映射常用方法

列举几个不常见但有用的方法

 |dict | defaultdict | OrderedDict | 说明
---|---|---|---
d.default_factory| |•| |在 __missing__ 函数中被调用的函数，用以给未找到的元素设置值
d.fromkeys(it,[initial]) |•| •| •|将迭代器 it 里的元素设置为映射里的键，如果有initial 参数，就把它作为这些键对应的值（默认是None）
d.get(k,[default])| •| •| •|返回键 k 对应的值，如果字典里没有键 k，则返回 None或者 default
d.pop(k, [defaul]| •| •| •|返回键 k 所对应的值，然后移除这个键值对。如果没有这个键，返回 None 或者 defaul
d.popitem()| •| •| •| 随机返回一个键值对并从字典里移除它
d.setdefault(k,[default])| •| •| •|若字典里有键k，则把它对应的值返回；若无，则让 d[k] = default，然后返回default

d.default_factory示例

In [45]:
from collections import defaultdict
dic = defaultdict(lambda:0)
dic[0]

0

d.fromkeys(it,[initial])示例

In [46]:
keys = range(0,10)
dic = {}
dic.fromkeys(keys, 0)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}

d.get(k,[default])示例

In [47]:
dic = {1:'a'}
print(dic.get(2,'b'))
print(dic.get(1, 'b'))

b
a


d.setdefault(k,[default]) 用setdefault处理找不到的键

In [50]:
dic = {}
dic.setdefault(0,[]).append('0')
dic.setdefault(0,[]).append('00')
dic

{0: ['0', '00']}

# 4. 映射的弹性键查询

有时候为了方便起见，就算某个键在映射里不存在，我们也希望在通过这个键读取值的时
候能得到一个默认值。有两个途径能帮我们达到这个目的，一个是通过 defaultdict 这
个类型而不是普通的 dict，另一个是给自己定义一个 dict 的子类，然后在子类中实现
\__missing\__ 方法。

* defaultdict  
defaultdict 里的 default_factory 只会在 \__getitem\__ 里被调用，在
其他的方法里完全不会发挥作用。比如， dd 是个 defaultdict， k 是个找不到的
键， dd[k] 这个表达式会调用 default_factory 创造某个默认值，而 dd.get(k)
则会返回 None。  
所有这一切背后的功臣其实是特殊方法 \__missing\__。它会在 defaultdict 遇到找不到
的键的时候调用 default_factory，而实际上这个特性是所有映射类型都可以选择去支
持的。

In [51]:
from collections import defaultdict
dic = defaultdict(list)
dic[0].append('0')
dic[0].append('00')
print(dic)

defaultdict(<class 'list'>, {0: ['0', '00']})


In [53]:
print(dic.get(1))

None


* 特殊方法\__missing\__   
\__missing\__ 方法只会被 \__getitem\__ 调用（比如在表达式 d[k] 中）。
提供 \__missing\__ 方法对 get 或者 \__contains\__（ in 运算符会用到这个方法）这
些方法的使用没有影响。这也是我在上一节最后的警告中提到， defaultdict 中的
default_factory 只对 \__getitem\__ 有作用的原因。

In [60]:
class MyDict(dict):
    def __missing__(self, key):
        print('miss key '+repr(key))
        raise KeyError(key)
dic = MyDict()

In [73]:
try:
    dic[0]
except Exception as x:
    print(repr(x))

miss key 0
KeyError(0,)


In [62]:
dic.get(0)

In [63]:
0 in dic

False

# 5. 字典的变种

* collections.OrderedDict  
这个类型在添加键的时候会保持顺序，因此键的迭代次序总是一致
的。 OrderedDict 的 popitem 方法默认删除并返回的是字典里的最后一个元素，但是如
果像 my_odict.popitem(last=False) 这样调用它，那么它删除并返回第一个被添加进
去的元素。
* collections.ChainMap  
该类型可以容纳数个不同的映射对象，然后在进行键查找操作的时候，这些对象会被
当作一个整体被逐个查找，直到键被找到为止。这个功能在给有嵌套作用域的语言做解释
器的时候很有用，可以用一个映射对象来代表一个作用域的上下文。
* collections.Counter  
这个映射类型会给键准备一个整数计数器。每次更新一个键的时候都会增加这个计数
器。所以这个类型可以用来给可散列表对象计数，或者是当成多重集来用——多重集合就
是集合里的元素可以出现不止一次。
* colllections.UserDict  
UserDict 是让用户继承写子类的。

In [112]:
from collections import Counter
ct = Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'c': 1, 'd': 1, 'r': 2})

# 6. 子类化UserDict

自定义字典我们不应该继承Dict，应该继承collections.UserDict  
Dict有时会在某些方法的实现上走一些捷径，导致我们不得不在它的子类中重写这些方法.  
UserDict 有一个叫作 data 的属性，是 dict 的实例，这个属性实际上是 UserDict 最终存储数据的地方。UserDict 的子类就能在实现 \__setitem\__之类方法时避免不必要的递归

UserDict继承于MutableMapping，UserDict的源码如下：

```python
class UserDict(MutableMapping):
    # Start by filling-out the abstract methods
    def __init__(*args, **kwargs):
        if not args:
            raise TypeError("descriptor '__init__' of 'UserDict' object "
                            "needs an argument")
        self, *args = args
        if len(args) > 1:
            raise TypeError('expected at most 1 arguments, got %d' % len(args))
        if args:
            dict = args[0]
        elif 'dict' in kwargs:
            dict = kwargs.pop('dict')
            import warnings
            warnings.warn("Passing 'dict' as keyword argument is deprecated",
                          PendingDeprecationWarning, stacklevel=2)
        else:
            dict = None
        self.data = {}
        if dict is not None:
            self.update(dict)
        if len(kwargs):
            self.update(kwargs)
    def __len__(self): return len(self.data)
    def __getitem__(self, key):
        if key in self.data:
            return self.data[key]
        if hasattr(self.__class__, "__missing__"):
            return self.__class__.__missing__(self, key)
        raise KeyError(key)
    def __setitem__(self, key, item): self.data[key] = item
    def __delitem__(self, key): del self.data[key]
    def __iter__(self):
        return iter(self.data)

    # Modify __contains__ to work correctly when __missing__ is present
    def __contains__(self, key):
        return key in self.data

    # Now, add the methods in dicts but not in MutableMapping
    def __repr__(self): return repr(self.data)
    def copy(self):
        if self.__class__ is UserDict:
            return UserDict(self.data.copy())
        import copy
        data = self.data
        try:
            self.data = {}
            c = copy.copy(self)
        finally:
            self.data = data
        c.update(self)
        return c
    @classmethod
    def fromkeys(cls, iterable, value=None):
        d = cls()
        for key in iterable:
            d[key] = value
        return d
```

In [93]:
from collections import UserDict
class MyDict2(UserDict):
    def __missing__(self, key):
        print('miss key '+repr(key))
        raise KeyError(key)
dic = MyDict2()

In [94]:
try:
    dic[0]
except Exception as x:
    print(repr(x))

miss key 0
KeyError(0,)


In [99]:
dic.get(0)

miss key 0


可以看到get方法也会调用我们定义的missing函数了，因为UserDict的get函数继承Mapping，Mapping的get源码如下：

```python
def get(self, key, default=None):
    'D.get(k[,d]) -> D[k] if k in D, else d.  d defaults to None.'
    try:
        return self[key]
    except KeyError:
        return default
```

# 7. 不可变映射类型

types 模块中引入了一个封装类名叫 MappingProxyType。如果给这个类一个映射，它会返回一个只读的映射视图。虽然是个只读视图，但是它是动态的。

In [2]:
from types import MappingProxyType
dic = {1:'1',2:'2'}
d_proxy = MappingProxyType(dic)
d_proxy

mappingproxy({1: '1', 2: '2'})

修改MappingProxyType会报错

In [4]:
# d_proxy[1]='11'

修改原dic对应的MappingProxyType会得到改变

In [6]:
dic[1]='11'
d_proxy

mappingproxy({1: '11', 2: '2'})

# 8. 集合论

- 集合的本质是许多唯一对象的聚集。
- 集合中的元素必须是可散列的， set 类型本身是不可散列的，但是 frozenset 可以。
- a | b 返回的是它们的合集， a & b 得到的是交集，而 a - b 得到的是差集。

空集合不能使用{}，否则初始化的是空字典

In [9]:
set_ = {1,2,3}
print(type(set_))
set_2 = {}
print(type(set_2))

<class 'set'>
<class 'dict'>


类似的也有集合推导式

In [10]:
set_ = {i for i in range(10)}
print(set_)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


集合的运算  
中缀运算符需要两侧的被操作对象都是集合类型，但是其他的所有方法则只要求所传入的参数是可迭代对象。

In [11]:
set1 = {1,2,3,4,5}
set2 = {4,5,6,7,8}
l = [7,8,9]

In [13]:
set1 & set2

{4, 5}

In [14]:
set1 | set2

{1, 2, 3, 4, 5, 6, 7, 8}

In [15]:
set1 - set2

{1, 2, 3}

In [16]:
set1 ^ set2

{1, 2, 3, 6, 7, 8}

In [19]:
set1.union(l)

{1, 2, 3, 4, 5, 7, 8, 9}

# 9. dict和set的背后

字典中的散列表

散列表其实是一个稀疏数组（总是有空白元素的数组称为稀疏数组）因为 Python 会设法保证大概还有三分之一的表元是空的，所以在快要达到这个阈值的时
候，原有的散列表会被复制到一个更大的空间里面。如果要把一个对象放入散列表，那么首先要计算这个元素键的散列值。 Python 中可以用
hash() 方法来做这件事情，

内置的 hash() 方法可以用于所有的内置类型对象。如果是自定义对象调用 hash()
的话，实际上运行的是自定义的 \__hash\__。如果两个对象在比较的时候是相等的，
那它们的散列值必须相等，否则散列表就不能正常运行了。  
为了让散列值能够胜任散列表索引这一角色，它们必须在索引空间中尽量分散开来。
这意味着在最理想的状况下，越是相似但不相等的对象，它们散列值的差别应该越
大。

一个可散列的对象必须满足以下要求。  
- 支持 hash() 函数，并且通过 \__hash\__() 方法所得到的散列值是不变的。
- 支持通过 \__eq\__() 方法来检测相等性。
- 若 a == b 为真，则 hash(a) == hash(b) 也为真。

字典在内存上的开销巨大。  
用元组取代字典就能节省空间的原因有两个：其一是避免了散列表所耗费的
空间，其二是无需把记录中字段的名字在每个元素里都存一遍。  
在用户自定义的类型中， __slots__ 属性可以改变实例属性的存储方式，由 dict 变
成 tuple，

set 和 frozenset 的实现也依赖散列表，但在它们的散列表里存放的只有元素的引用
（就像在字典里只存放键而没有相应的值）。在 set 加入到 Python 之前，我们都是把字
典加上无意义的值当作集合来用的。  
- 集合里的元素必须是可散列的。
- 集合很消耗内存。
- 可以很高效地判断元素是否存在于某个集合。
- 元素的次序取决于被添加到集合里的次序。
- 往集合里添加元素，可能会改变集合里已有元素的次序。